In [1]:
import h5py
import os
import numpy as np

from keras.models import Model
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


First, let's load the training and test data set that we created earlier. We also set a flag for whether the expected values are one-hot encoded classes or the range of coffee pot fullness (0.0 - 1.0).

In [2]:
is_regression = False

In [3]:
data_save_path = '../../data/coffee_video/label_data/v3'
class_data_set_name='coffee_class_data_v3.h5'
regression_data_set_name='coffee_regression_data_v3.h5'

In [4]:
def get_data(filepath):
    data_file = h5py.File(filepath,'r')

    train_data = data_file['train_data'][:]
    train_labels = data_file['train_labels'][:]

    test_data = data_file['test_data'][:]
    test_labels = data_file['test_labels'][:]

    return train_data, train_labels, test_data, test_labels

if(is_regression):
    filepath=os.path.join(data_save_path,regression_data_set_name)
else:
    filepath=os.path.join(data_save_path,class_data_set_name)

train_data, train_labels, test_data, test_labels  = get_data(filepath)

print('Training Data Shape:',train_data.shape)
print('Training Labels Shape:',train_labels.shape)

print('Test Data Shape:',test_data.shape)
print('Test Labels Shape:',test_labels.shape)

Training Data Shape: (1848, 540, 960, 3)
Training Labels Shape: (1848, 6)
Test Data Shape: (462, 540, 960, 3)
Test Labels Shape: (462, 6)


Now, let's define our model. Using keras, we define the first layer of the model based on the image dimensions, which would be (width, height, channels). After this, we add a series of 2D convolutional layers which consists of a covolution operation, a non-linear activation, and then a max pooling layer. Hyperparameters for the number of filters, filter size, etc. are defined in a list for each layer. The convolutional layers act as spatial feature detectors which learn different feature representations which will be used by the next steps.

Then, the output is flattened, and we add a number of dense fully connected layers that are also defined as a list. Each layer consists of the numbber of hidden units,  

Finally, we add the output layer. If we are using this as a multi-class problem (e.g. varying amounts of coffee pot fullness are bucketed and each is treated as a separate class), then our final output layers consists of a neuron for each class, with softmax activation. Otherwise, in the regression case (where we treat the output as a continuous variable), we add a dense layer with a single node which would output values betwen the range of 0-1. Mean squared error is then used as the loss function to train the network.

The model is defined in a JSON file. First, let's load the configuration.

In [5]:
import build_ishame_model

iShame_model_config = 'iShameModel.json'
conv_layers, conn_layers = build_ishame_model.load_model_def(iShame_model_config)
print("conv_layers",conv_layers)
print("conn_layers",conn_layers)

conv_layers [{'num_filters': 16, 'filter_size': '(5,5)', 'stride_size': '(2,2)', 'activation': 'relu', 'pool_size': '(2,2)'}, {'num_filters': 32, 'filter_size': '(5,5)', 'stride_size': '(2,2)', 'activation': 'relu', 'pool_size': '(2,2)'}, {'num_filters': 64, 'filter_size': '(5,5)', 'stride_size': '(2,2)', 'activation': 'relu', 'pool_size': '(2,2)'}]
conn_layers [{'hidden_units': 20, 'activation': 'relu', 'dropout': 0.5}, {'hidden_units': 10, 'activation': 'relu'}]


Now let's define some additional hyperparameters for our model, based on whether we are treating this as a classification or regression problem.

In [6]:
input_shape = train_data.shape[1:]
if(is_regression):
    num_classes=None
else:
    num_classes=train_labels.shape[-1]
batch_size=32
epochs=10

We're now ready to build our model.

In [7]:
model = build_ishame_model.build_model(input_shape=input_shape, \
                                        conv_layers=conv_layers, \
                                        conn_layers=conn_layers, \
                                        output_classes=num_classes)
build_ishame_model.compile_model(model, is_regression)

Let's take a look at the different layers. We can the convolutional and fully connected layers defined in the config have been successfully created. 

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_0 (Conv2D)              (None, 268, 478, 16)      1216      
_________________________________________________________________
max_pool_0 (MaxPooling2D)    (None, 134, 239, 16)      0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 65, 118, 32)       12832     
_________________________________________________________________
max_pool_1 (MaxPooling2D)    (None, 32, 59, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 14, 28, 64)        51264     
_________________________________________________________________
max_pool_2 (MaxPooling2D)    (None, 7, 14, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)             

In [12]:
plot_model(model)

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

Now, we train our model with the given training data and hyperparameters.

In [11]:
model.fit(train_data, train_labels, epochs = epochs, batch_size = batch_size)

Epoch 1/10
1848/1848 [==============================] - 596s 323ms/step - loss: 1.6420 - categorical_accuracy: 0.2744
Epoch 2/10
1848/1848 [==============================] - 532s 288ms/step - loss: 1.3636 - categorical_accuracy: 0.3409
Epoch 3/10
1848/1848 [==============================] - 375s 203ms/step - loss: 1.2751 - categorical_accuracy: 0.3772
Epoch 4/10
1848/1848 [==============================] - 360s 195ms/step - loss: 1.1872 - categorical_accuracy: 0.4616
Epoch 5/10
1848/1848 [==============================] - 365s 198ms/step - loss: 1.1468 - categorical_accuracy: 0.4816
Epoch 6/10
1848/1848 [==============================] - 367s 199ms/step - loss: 1.1024 - categorical_accuracy: 0.4957
Epoch 7/10
1848/1848 [==============================] - 390s 211ms/step - loss: 1.0387 - categorical_accuracy: 0.5281
Epoch 8/10
1848/1848 [==============================] - 397s 215ms/step - loss: 1.0109 - categorical_accuracy: 0.5455
Epoch 9/10
1848/1848 [==============================] - 


Let's verify our model performance by checking our performance on our holdout set.

In [13]:
results = model.evaluate(test_data, test_labels)
print("Loss:",results[0])
print("Metric:",results[1])

462/462 [==============================] - 68s 147ms/step
Loss: 0.643123876093786
Metric: 0.8333333134651184


In [15]:
model_save_path = '../../data/coffee_video/model'

if(is_regression):
    model_weights_file='coffee_regression_v3_sequential_weights.h5'
else:
    model_weights_file='coffee_class_v3_sequential_weights.h5'

model.save_weights(os.path.join(model_save_path,model_weights_file))
